### 안경씌우기
1. 얼굴을 탐지한다.
2. 안경을 씌우기 위해 눈의 위치를 탐색한다.
3. 안경을 씌우는 위치에 안경에 해당하는 이미지를 사이즈 조정하여 넣는다. 비트 연산하여 넣는다. 
4. 안경이 씌워진 얼굴 이미지를 출력한다. 
5. 추가 : 눈의 각도를 계산해서 안경 이미지도 회전하는 코드를 작성해본다.

In [2]:
import cv2
import mediapipe as mp
import numpy as np

In [3]:
# 안경 이미지 로드
glasses_img = cv2.imread('glasses.png', cv2.IMREAD_UNCHANGED)  # 알파 채널을 포함하여 로드
print(glasses_img.shape)
cv2.imshow("glass_img", glasses_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

(353, 707, 4)


### 1. 고정된 위치에 안경 올려보기

In [6]:
# MediaPipe 얼굴 검출 초기화
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.5)

glasses_img = cv2.imread('glasses.png', cv2.IMREAD_UNCHANGED)  # 알파 채널을 포함하여 로드

# 웹캠 캡처 객체 생성
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break  # 프레임을 제대로 읽지 못하면 루프 탈출

    # BGR 이미지를 RGB로 변환
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # 얼굴 랜드마크 검출
    results = face_mesh.process(rgb_frame)

    # 얼굴 랜드마크에 따라 안경 오버레이
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # 눈의 위치를 기준으로 안경 위치 조정 및 크기 조정 로직 필요 (여기서는 간단히 처리)
            # 예시로 랜드마크를 사용하지 않고 고정된 위치에 적용하는 방식으로 단순화
            # 실제 사용시에는 랜드마크를 기반으로 안경의 위치와 크기를 동적으로 조정해야 합니다.
            
            # 안경 이미지의 알파 채널과 RGB 채널 분리
            glasses_alpha = glasses_img[:, :, 3] / 255.0
            glasses_color = glasses_img[:, :, :3]

            # 안경을 적용할 위치 계산 (여기서는 전체 이미지 크기를 기준으로 간단히 설정)
            x, y, w, h = 100, 100, 200, 100  # 예시 위치 및 크기
            glasses_resized = cv2.resize(glasses_color, (w, h))
            alpha_resized = cv2.resize(glasses_alpha, (w, h), interpolation=cv2.INTER_AREA)

            # 알파 블렌딩을 통한 안경 이미지 합성
            for c in range(0, 3):
                frame[y:y+h, x:x+w, c] = alpha_resized * glasses_resized[:, :, c] + (1-alpha_resized) * frame[y:y+h, x:x+w, c]

    # 결과 표시
    cv2.imshow('Glasses Overlay', frame)

    # 'q'를 누르면 루프 탈출
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


### 2. 눈의 위치를 계산하여 안경 씌어보기

In [5]:
# MediaPipe Face Mesh 초기화
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.5)

# 웹캠 캡처 객체 생성
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break  # 프레임을 제대로 읽지 못하면 루프 탈출

    # BGR 이미지를 RGB로 변환
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # 얼굴 랜드마크 검출
    results = face_mesh.process(rgb_frame)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            h, w, _ = frame.shape
            # 랜드마크 인덱스: 왼쪽 눈 362, 오른쪽 눈 133
            left_eye = face_landmarks.landmark[362]
            right_eye = face_landmarks.landmark[133]

            # 눈의 중심 위치 계산
            eye_center_x = int((left_eye.x + right_eye.x) / 2 * w)
            eye_center_y = int((left_eye.y + right_eye.y) / 2 * h)

            # 두 눈 사이의 x, y 좌표 차이 계산
            delta_x = left_eye.x - right_eye.x
            delta_y = left_eye.y - right_eye.y

            # 안경 이미지의 크기 조정 (여기서는 두 눈 사이의 거리를 기준으로 조정합니다)
            eye_distance = int(np.linalg.norm(np.array([delta_x, delta_y]) * [w, h]))
            glasses_width = int(eye_distance * 2.7) # 안경 크기가 너무 작게 표시된다면 이 값을 수정
            scale_factor = glasses_width / glasses_img.shape[1]
            resized_glasses = cv2.resize(glasses_img, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_AREA)

            # 안경 이미지의 알파 채널과 RGB 채널 분리
            glasses_alpha = resized_glasses[:, :, 3] / 255.0
            glasses_color = resized_glasses[:, :, :3]

            # 안경 위치 조정
            x = eye_center_x - glasses_width // 2
            y = eye_center_y - resized_glasses.shape[0] // 2

            # 알파 블렌딩을 통한 안경 이미지 합성
            for c in range(0, 3):
                frame[y:y+resized_glasses.shape[0], x:x+glasses_width, c] = glasses_alpha * glasses_color[:, :, c] + (1-glasses_alpha) * frame[y:y+resized_glasses.shape[0], x:x+glasses_width, c]
            
            # 아래와 같이 해도 되지만.. 정수형은 정확하지 않습니다.
            # glasses_alpha = resized_glasses[:, :, 3]
            # mask_inv = cv2.bitwise_not(glasses_alpha)
            # resized_glasses_rgb = resized_glasses[:, :, :3]

            # # 배경 영역을 준비합니다.
            # background = cv2.bitwise_and(frame[y:y+resized_glasses.shape[0], x:x+glasses_width], frame[y:y+resized_glasses.shape[0], x:x+glasses_width], mask=mask_inv)

            # # 안경 영역을 준비합니다.
            # glasses_area = cv2.bitwise_and(resized_glasses_rgb, resized_glasses_rgb, mask=glasses_alpha)

            # # 안경 영역과 배경 영역을 합성합니다.
            # combined = cv2.add(background, glasses_area)

            # # 합성된 이미지를 원본 이미지에 적용합니다.
            # frame[y:y+resized_glasses.shape[0], x:x+glasses_width] = combined

    # 결과 표시
    cv2.imshow('Glasses Overlay', frame)

    # 'q'를 누르면 루프 탈출
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


### 3. 눈의 회전까지 고려해보기

In [2]:
# MediaPipe Face Mesh 초기화
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.5)

# 안경 이미지 로드
glasses_img = cv2.imread('glasses.png', cv2.IMREAD_UNCHANGED)  # 알파 채널을 포함하여 로드

# 웹캠 캡처 객체 생성
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break  # 프레임을 제대로 읽지 못하면 루프 탈출

    # BGR 이미지를 RGB로 변환
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # 얼굴 랜드마크 검출
    results = face_mesh.process(rgb_frame)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            h, w, _ = frame.shape
            # 랜드마크 인덱스: 왼쪽 눈 362, 오른쪽 눈 133
            left_eye = face_landmarks.landmark[362]
            right_eye = face_landmarks.landmark[133]

            # 두 눈 사이의 x, y 좌표 차이 계산
            delta_x = left_eye.x - right_eye.x
            delta_y = left_eye.y - right_eye.y

            # 두 눈이 이루는 각도 계산
            angle_radians = np.arctan2(delta_y, delta_x)  # 라디안 단위의 각도
            angle_degrees = np.degrees(angle_radians)  # 도 단위로 변환

            # 눈의 중심 위치 계산
            eye_center_x = int((left_eye.x + right_eye.x) / 2 * w)
            eye_center_y = int((left_eye.y + right_eye.y) / 2 * h)
            
            # 이미지 회전
            h, w = glasses_img.shape[:2]
            center = (w // 2, h // 2)
            M = cv2.getRotationMatrix2D(center, -angle_degrees, 1)

            # # 회전할 때 이미지가 잘린다면 추가하세요
            # cos = np.abs(M[0, 0])
            # sin = np.abs(M[0, 1])

            # w = int((h * sin) + (w * cos))
            # h = int((h * cos) + (w * sin))

            # # 변환 행렬의 이동 부분 조정
            # M[0, 2] += (w / 2) - center[0]
            # M[1, 2] += (h / 2) - center[1]

            rotated_glasses_img = cv2.warpAffine(glasses_img, M, (w, h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)

            # 안경 이미지의 크기 조정 (여기서는 두 눈 사이의 거리를 기준으로 조정합니다)
            eye_distance = int(np.linalg.norm(np.array([delta_x, delta_y]) * [w, h]))
            glasses_width = int(eye_distance * 2.7) # 안경 크기가 너무 작게 표시된다면 이 값을 수정
            scale_factor = glasses_width / rotated_glasses_img.shape[1]
            resized_glasses = cv2.resize(rotated_glasses_img, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_AREA)

            # 안경 이미지의 알파 채널과 RGB 채널 분리
            glasses_alpha = resized_glasses[:, :, 3] / 255.0
            glasses_color = resized_glasses[:, :, :3]

            # 안경 위치 조정
            x = eye_center_x - glasses_width // 2
            y = eye_center_y - resized_glasses.shape[0] // 2

            # 알파 블렌딩을 통한 안경 이미지 합성
            for c in range(0, 3):
                frame[y:y+resized_glasses.shape[0], x:x+glasses_width, c] = glasses_alpha * glasses_color[:, :, c] + (1-glasses_alpha) * frame[y:y+resized_glasses.shape[0], x:x+glasses_width, c]

    # 결과 표시
    cv2.imshow('Glasses Overlay', frame)

    # 'q'를 누르면 루프 탈출
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()